In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm

%matplotlib inline
print('Se han importado todas las librerias')

Se han importado todas las librerias


In [7]:
df_test = pd.read_parquet('test.parquet')
df_test.isnull().sum()

id                             0
url                            0
region                         0
region_url                     0
type                           0
sqfeet                         0
beds                           0
baths                          0
cats_allowed                   0
dogs_allowed                   0
smoking_allowed                0
wheelchair_access              0
electric_vehicle_charge        0
comes_furnished                0
laundry_options             7855
parking_options            14005
image_url                      0
description                    0
lat                          196
long                         196
state                          0
dtype: int64

#### Eliminar las columnas que tienen nulos

In [8]:
df_test.dropna(axis=1,inplace=True)

In [9]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38498 entries, 0 to 38497
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       38498 non-null  int64  
 1   url                      38498 non-null  object 
 2   region                   38498 non-null  object 
 3   region_url               38498 non-null  object 
 4   type                     38498 non-null  object 
 5   sqfeet                   38498 non-null  int64  
 6   beds                     38498 non-null  int64  
 7   baths                    38498 non-null  float64
 8   cats_allowed             38498 non-null  int64  
 9   dogs_allowed             38498 non-null  int64  
 10  smoking_allowed          38498 non-null  int64  
 11  wheelchair_access        38498 non-null  int64  
 12  electric_vehicle_charge  38498 non-null  int64  
 13  comes_furnished          38498 non-null  int64  
 14  image_url             

#### Eliminar columnas que no son importante para el analisis ya que no tienen importancia logica

In [10]:
'''Se procedio a revisar 3 registros aleatorios de las url y generaba error 404,
por ello, se eliminan las columnas que a primera vista no son importante
para el modelo,'''

print('EL dataframe tiene',len(df_test.columns), 'columnas')
df_test.drop(['url','region_url','image_url','description'], axis=1, inplace=True)
print('EL dataframe tiene',len(df_test.columns), 'columnas')

EL dataframe tiene 17 columnas
EL dataframe tiene 13 columnas


In [11]:
df_test.head(1)

,id,region,type,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,state
0,7037609789,ann arbor,manufactured,1344,3,2.0,0,0,1,0,0,0,mi


In [12]:
#Label Encoder df_test
#region
label_enc = LabelEncoder()
df_test['region'] =label_enc.fit_transform(df_test['region'])
print('Etiqueta de region:\n',list(label_enc.classes_))

#state
label_enc = LabelEncoder()
df_test['state'] =label_enc.fit_transform(df_test['state'])
print('etiquetas de state:\n',list(label_enc.classes_))

#type
label_enc = LabelEncoder()
df_test['type'] =label_enc.fit_transform(df_test['type'])
print('etiquetas de type:\n',list(label_enc.classes_))

Etiqueta de region:
 ['SF bay area', 'abilene', 'akron / canton', 'albany', 'albuquerque', 'altoona-johnstown', 'amarillo', 'ames', 'anchorage / mat-su', 'ann arbor', 'annapolis', 'appleton-oshkosh-FDL', 'asheville', 'ashtabula', 'athens', 'atlanta', 'auburn', 'augusta', 'austin', 'bakersfield', 'baltimore', 'baton rouge', 'battle creek', 'beaumont / port arthur', 'bellingham', 'bemidji', 'bend', 'billings', 'binghamton', 'birmingham', 'bismarck', 'bloomington', 'bloomington-normal', 'boise', 'boone', 'boston', 'boulder', 'bowling green', 'bozeman', 'brainerd', 'brownsville', 'brunswick', 'buffalo', 'butte', 'cape cod / islands', 'catskills', 'cedar rapids', 'central NJ', 'central louisiana', 'central michigan', 'champaign urbana', 'charleston', 'charlotte', 'charlottesville', 'chattanooga', 'chautauqua', 'chicago', 'chico', 'chillicothe', 'cincinnati', 'clarksville', 'cleveland', 'clovis / portales', 'college station', 'colorado springs', 'columbia', 'columbia / jeff city', 'columbus'

### Hacer la Correlacion para elegir las varibles
Se toman las columnas con valores de 0.1 

In [13]:
corr = df_test.corr()
corr

,id,region,type,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,state
id,1.000000,-0.008103,-0.041622,-0.021631,-0.005766,-0.023515,0.042138,0.034533,0.022069,0.008597,0.019346,-0.030870,0.002974
region,-0.008103,1.000000,0.022210,0.005583,-0.006350,0.005480,0.030108,0.019724,-0.012775,-0.014419,0.011639,-0.024253,-0.012599
type,-0.041622,0.022210,1.000000,0.219172,0.056637,0.205053,-0.099505,-0.064111,-0.067119,-0.042847,0.003355,0.032508,0.011704
sqfeet,-0.021631,0.005583,0.219172,1.000000,0.065701,0.317127,-0.055366,-0.031395,-0.024935,-0.012392,0.004852,0.002039,-0.017891
beds,-0.005766,-0.006350,0.056637,0.065701,1.000000,0.613230,-0.020588,-0.014766,0.001944,-0.008532,-0.002996,-0.000578,-0.003610
baths,-0.023515,0.005480,0.205053,0.317127,0.613230,1.000000,-0.022616,0.007578,-0.014843,0.012069,0.004583,0.029731,-0.024067
cats_allowed,0.042138,0.030108,-0.099505,-0.055366,-0.020588,-0.022616,1.000000,0.886800,0.018572,0.123087,0.050117,-0.053430,0.030380
dogs_allowed,0.034533,0.019724,-0.064111,-0.031395,-0.014766,0.007578,0.886800,1.000000,0.014378,0.130026,0.050109,-0.032931,0.018677
smoking_allowed,0.022069,-0.012775,-0.067119,-0.024935,0.001944,-0.014843,0.018572,0.014378,1.000000,-0.223166,-0.094171,-0.156620,-0.018396
wheelchair_access,0.008597,-0.014419,-0.042847,-0.012392,-0.008532,0.012069,0.123087,0.130026,-0.223166,1.000000,0.197551,0.198187,0.025126
